In [1]:
import gradio as gr

# ---------- Page 1: 3 Buttons ----------
def button_clicked(choice):
    return f"You clicked: {choice}"

with gr.Blocks() as page1:
    gr.Markdown("## Page 1: Choose a Button")
    with gr.Row():
        btn1 = gr.Button("Button 1")
        btn2 = gr.Button("Button 2")
        btn3 = gr.Button("Button 3")
    output1 = gr.Textbox(label="Result")
    
    btn1.click(fn=lambda: button_clicked("Button 1"), outputs=output1)
    btn2.click(fn=lambda: button_clicked("Button 2"), outputs=output1)
    btn3.click(fn=lambda: button_clicked("Button 3"), outputs=output1)

# ---------- Page 2: Simple Chat ----------
chat_history = []

def chat_response(message, history):
    history = history or []
    response = f"You said: {message}"
    history.append((message, response))
    return "", history

with gr.Blocks() as page2:
    gr.Markdown("## Page 2: Chat with the App")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Message")
    send_btn = gr.Button("Send")
    
    send_btn.click(fn=chat_response, inputs=[msg, chatbot], outputs=[msg, chatbot])

# ---------- Page 3: Dummy Text ----------
with gr.Blocks() as page3:
    gr.Markdown("## Page 3: Dummy Text")
    gr.Textbox(value="This is some dummy text. Hello world!", lines=5, label="Note")

# ---------- Main App with Tabs ----------
with gr.Blocks() as demo:
    with gr.Tab("Buttons Page"):
        page1.render()
    with gr.Tab("Chat Page"):
        page2.render()
    with gr.Tab("Text Page"):
        page3.render()

# Launch the app
demo.launch()


c:\Users\Arian\Desktop\doc-chat\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Arian\AppData\Local\Temp\ipykernel_27204\450893732.py:30: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
